In [ ]:
# Install/import dependencies
# prophet, pandas

import warnings
warnings.simplefilter('ignore')

import pandas as pd
from prophet import Prophet


In [ ]:
# Read data + process data 
from pandas import read_csv
path = 'data.Prophet.csv'
df = read_csv(path)
# First few rows
print(df.head())

In [ ]:
# ds is date, y is values

df.dtypes



 ds is date, y is values

df.dtypes

In [ ]:
# Pre-processing

df['Year'] = df['Time Date'].apply(lambda x: str(x)[-8:-4])
df['Month'] = df['Time Date'].apply(lambda x: str(x)[-4:-2])
df['Day'] = df['Time Date'].apply(lambda x: str(x)[-2:])
df['ds'] = pd.DatetimeIndex(df['Year'] +'-'+df['Month']+'-'+df['Day'])

In [ ]:
# Check to see if it is datetime
df.dtypes

In [ ]:
df.head

In [ ]:
# Drop columns that are not needed inc. unique values, axis=1 is to drop the coloumns 
# inplace=True is for the same dataset (not creating another dataset)
# Can filter by specific type of emergency department by using a slicer (df[df['X']=='specifics'])

df.drop(['Time Date', 'Year', 'Month', 'Day'], axis=1, inplace=True)
df.columns = 'y', 'ds'

In [ ]:
df.head

In [ ]:
df = df.groupby('ds')['y'].sum()
# Print the output.
md = df.reset_index()
print(md)

In [ ]:
# Train Model

m = Prophet(interval_width=0.95)

model = m.fit(md)

In [ ]:
#Forecast - trends, lower and upper estimates, daily seasonality, yhat is the predicted forecast
#periods you want to forecast for - up to 100. Frequency is by Month
# second line forecasts forward

future = m.make_future_dataframe(periods=20,freq='M')
forecast = m.predict(future)
forecast.head()

In [ ]:
#Plot data, prophet model is stored in variable m


plot1 = m.plot(forecast)
plt.title('Monthly Sales')
plt.ylabel('Total Sales')
plt.xlabel('Date')
plt.ticklabel_format(style='plain', axis='y')
plt.legend()
plt.show()

In [ ]:
#Decompose

plot2 = m.plot_components(forecast)

In [ ]:
#Ceate a metric dataframe that essentially joins the actual values from monthly_inc (y)
#with the predicted values from the forecast table
#using the common key 'ds'
metric_df = forecast.set_index('ds')[['yhat']].join(md.set_index('ds').y).reset_index()
metric_df

In [ ]:
#let's drop those predicted values and focus on how well the model has fit with historical data
metric_df.dropna(inplace=True)

In [ ]:
#RMSE 
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse_prophet = sqrt(mean_squared_error(metric_df.y, metric_df.yhat))
print(rmse_prophet)